<a href="https://colab.research.google.com/github/BahatiKilongo/GDA_Live_coding_FML23/blob/main/BAHATIKILONGO__GDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# GDA Implementation.

Implement the Gaussian Discriminant Analysis (GDA) learning algorithm following the steps as discussed in class.

INSTRUCTION: Rename your notebook as: <br>
`firstName_LastName_Live_coding_GDA.ipynb`.

Notes: 
* Do not use any built-in functions to complete a task;
* Do not import additional libraries.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import make_classification

In [ ]:
# Generate data
def generate_data():
  x, y = make_classification(n_samples= 1000, n_features=3, n_redundant=0, 
                           n_informative=3, random_state=1, 
                           n_clusters_per_class=1)
  
  return x,y

x,y= generate_data() # get data
print(x.shape, y.shape)

(1000, 3) (1000,)


In [ ]:

def split_data(X,y, train_size = 0.8):
   
  n = int(len(X)*train_size)
  indices = np.arange(len(X))
  np.random.shuffle(indices)
  train_idx = indices[: n]
  test_idx = indices[n:]
  X_train, y_train = X[train_idx], y[train_idx]
  X_test, y_test = X[test_idx], y[test_idx]

  return X_train, y_train, X_test, y_test


In [ ]:
X_train,y_train, X_test, y_test= split_data(x,y) # split your data into x_train, x_test, y_train, y_test
print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

(800, 3) (800,) (200, 3) (200,)


In [ ]:
print(len(np.unique(y_train)))

2


In [ ]:
def covariance(x, mu):
  mu=np.mean(x, axis=0)
  n,d=x.shape
  cov=np.zeros((d,d))
  #print(cov.shape)
  # Easy way: cov= np.cov(x, rowvar=0) but do not use it. One can use it to assess his/her result.
  for i in range(d):
    for j in range(d):
      cov[i,j]=np.sum((x[:,i]-mu[i])*(x[:,j]-mu[j]))/(len(x)-1)
  return cov

In [ ]:
mu=np.mean(X_test, axis=0)
covariance(X_test,mu ) - np.cov(X_test, rowvar=0)

array([[ 1.55431223e-15,  0.00000000e+00,  4.44089210e-16],
       [ 0.00000000e+00,  2.22044605e-16, -1.56125113e-17],
       [ 4.44089210e-16, -1.56125113e-17,  1.11022302e-15]])

In [ ]:
class GDA:
  def __init__(self):
    ## set mu, phi and sigma to None
    self.mu = None
    self.phi = None
    self.sigma = None
    
  def fit(self,x,y):
    K = len(np.unique(y.reshape(-1))) # Number of class.
    d= x.shape[1]  # input dim
    m= x.shape[0] # Number of examples.
    #x = x.reshape(m,-1)

    
    
    ## Initialize mu, phi and sigma
    self.mu= np.zeros((K,d))       #: kxd, i.e., each row contains an individual class mu.
    self.sigma= np.zeros((K,d,d))  #: kxdxd, i.e., each row contains an individual class sigma.
    self.phi= np.zeros((K,1))      # k-dimension

    ## START THE LEARNING: estimate mu, phi and sigma.
    for i in range(K):
      class_ind = (i == y)
      #self.mu[i] = np.mean(x[class_ind,:], axis=0)
      self.mu[i] = np.sum(x[y == i]) *x[i] /np.sum(x[class_ind])
      self.sigma[i] = covariance(x[y==i],self.mu[i])
      self.phi[i] = (np.sum(y==1)) / m
    return self.mu, self.sigma, self.phi

      


  def predict_proba(self,x):
    
    # reshape or flatt x.
    #x= np.reshape(-1,1)
    d= x.shape[0]
    K =self.mu.shape[0] # Number of classes we have in our case it's k = 2
    proba=np.zeros((d,K))
    
    ## START THE LEARNING: estimate mu, phi and sigma.
    for i in range(K):
      sigma_det=np.linalg.det(self.sigma[i])
      sigma_inv=np.linalg.inv(self.sigma[i])
      for j in range(x.shape[0]):
        fact1 =1/((2*np.pi)**(d/2)*(sigma_det**0.5))
        fact2 =np.exp( -0.5*((x[j]-self.mu[i]).T)@(sigma_inv)@(x[j]-self.mu[i]) )
        gaussian=np.exp(fact2)
        proba[j, i]= gaussian*self.phi[i]
    return proba

  def predict(self,x):
    pred=self.predict_proba(x)
    ypred=np.argmax(pred, axis=1)
    return ypred
  
  def accuracy(self, y, ypreds):
    reslt=np.mean(y==ypreds)
    return reslt*100

In [ ]:
model = GDA()
model.fit(X_train, y_train)

(array([[ 1.98002276,  0.12186164,  0.63331114],
        [ 1.83390275, -1.06230907,  1.12266418]]),
 array([[[ 0.86158965, -0.33611909, -0.05787815],
         [-0.33611909,  1.6472537 ,  0.09882649],
         [-0.05787815,  0.09882649,  0.03919141]],
 
        [[ 0.79704476,  0.34257941,  0.09802025],
         [ 0.34257941,  0.35206826, -0.07220905],
         [ 0.09802025, -0.07220905,  1.56433587]]]),
 array([[0.505],
        [0.505]]))

In [ ]:
yproba = model.predict_proba(X_test)
yproba

array([[0.50566457, 0.50500005],
       [0.505     , 0.505     ],
       [0.505     , 0.505     ],
       [0.50733371, 0.505     ],
       [0.5663933 , 0.50500689],
       [0.78695532, 0.50645439],
       [0.505     , 0.505     ],
       [0.505     , 0.505     ],
       [0.58236906, 0.505     ],
       [0.50500126, 0.50507579],
       [0.50579847, 0.505     ],
       [0.52337307, 0.505     ],
       [0.63324032, 0.505796  ],
       [1.142297  , 0.50515808],
       [0.59300257, 0.7691339 ],
       [0.505     , 0.505     ],
       [0.51433578, 0.505     ],
       [0.50500347, 0.505     ],
       [0.50510796, 0.505     ],
       [0.87118862, 0.56699166],
       [0.505     , 0.505     ],
       [0.58535932, 0.505     ],
       [0.50913361, 0.505     ],
       [0.5326869 , 0.505     ],
       [0.505     , 0.505     ],
       [0.59941135, 0.7239743 ],
       [0.51235599, 0.50506219],
       [1.08384886, 0.50500533],
       [0.60857803, 0.5479582 ],
       [0.54837075, 0.505     ],
       [0.

In [ ]:
ypreds= model.predict(X_test)
ypreds



array([0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0,
       1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0,
       0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1,
       1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0,
       0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0,
       0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0,
       1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0])

In [ ]:
model.accuracy(y_test, ypreds)

67.0

#Logistic Regression

In [ ]:
class LogisticRegression:
  '''
  The goal of this class is to create a LogisticRegression class, 
  that we will use as our model to classify data point into a corresponding class
  '''
  def __init__(self,lr,n_epochs):
    self.lr = lr
    self.n_epochs = n_epochs
    self.train_losses = []
    self.w = None
    self.weight = []

  def add_ones(self, x):
    ##### WRITE YOUR CODE HERE #####
    one = np.ones((x.shape[0],1))
    return np.hstack((one,x))
    #### END CODE ####

  def sigmoid(self, x):
    return 1/(1+np.exp(-x@self.w))


  def cross_entropy(self, x, y_true):
    ##### WRITE YOUR CODE HERE #####
    y_pred = self.sigmoid(x)
    loss = -np.mean(y_true*np.log(y_pred)+(1-y_true)*np.log(1-y_pred))
    return loss
    #### END CODE ####
  
  def predict_proba(self,x):  #This function will use the sigmoid function to compute the probalities
    ##### WRITE YOUR CODE HERE #####
    x= self.add_ones(x)
    proba = self.sigmoid(x)
    return proba
    #### END CODE ####

  def predict(self,x):
    ##### WRITE YOUR CODE HERE #####
    probas = self.predict_proba(x)
    output = [0 if p<0.5 else 1 for p in probas]#np.where(probas>=0.5, 1, 0)      #convert the probalities into 0 and 1 by using a treshold=0.5
    return output
    #### END CODE ####

  def fit(self,x,y):
    # Add ones to x
    x=self.add_ones(x)

    # reshape y if needed
    y=y.reshape(-1,1)

    # Initialize w to zeros vector >>> (x.shape[1])
    self.w=np.zeros((x.shape[1],1))

    for epoch in range(self.n_epochs):
      # make predictions
      ypred = self.sigmoid(x)

      #compute the gradient
      dl = (-1/x.shape[0])*(x.T@(y-ypred))

      #update rule
      self.w=self.w-self.lr*dl

      #Compute and append the training loss in a list
      loss = self.cross_entropy(x,y)
      self.train_losses.append(loss)

      if epoch%1000 == 0:
        print(f'loss for epoch {epoch}  : {loss}')

  def accuracy(self,y_true, y_pred):
    ##### WRITE YOUR CODE HERE #####
    acc = np.mean(y_true==y_pred)*100
    return acc

In [ ]:
model = LogisticRegression(0.01,n_epochs=10000)
model.fit(X_train,y_train)

loss for epoch 0  : 0.6883420100966741
loss for epoch 1000  : 0.19939132988838457
loss for epoch 2000  : 0.1751950169548517
loss for epoch 3000  : 0.16616634669872749
loss for epoch 4000  : 0.16131500670259905
loss for epoch 5000  : 0.15822537170483092
loss for epoch 6000  : 0.15605733636588565
loss for epoch 7000  : 0.15444091528407455
loss for epoch 8000  : 0.15318651020708832
loss for epoch 9000  : 0.15218580544708607


In [ ]:
ypred_train = model.predict(X_train)
acc = model.accuracy(y_train,ypred_train)
print(f"The training accuracy is: {acc}")
print(" ")

ypred_test = model.predict(X_test)
acc = model.accuracy(y_test,ypred_test)
print(f"The test accuracy is: {acc}")

The training accuracy is: 50.005937499999995
 
The test accuracy is: 49.985
